![Shopping trolley in front of a laptop](./iStock-1249219777.jpg)

It's simple to buy any product with a click and have it delivered to your door. Online shopping has been rapidly evolving over the last few years, making our lives easier. But behind the scenes, e-commerce companies face a complex challenge that needs to be addressed. 

Uncertainty plays a big role in how the supply chains plan and organize their operations to ensure that the products are delivered on time. These uncertainties can lead to challenges such as stockouts, delayed deliveries, and increased operational costs.

You work for the Sales & Operations Planning (S&OP) team at a multinational e-commerce company. They need your help to assist in planning for the upcoming end-of-the-year sales. They want to use your insights to plan for promotional opportunities and manage their inventory. This effort is to ensure they have the right products in stock when needed and ensure their customers are satisfied with the prompt delivery to their doorstep.


## The Data

You are provided with a sales dataset to use. A summary and preview are provided below.

# Online Retail.csv

| Column     | Description              |
|------------|--------------------------|
| `'InvoiceNo'` | A 6-digit number uniquely assigned to each transaction |
| `'StockCode'` | A 5-digit number uniquely assigned to each distinct product |
| `'Description'` | The product name |
| `'Quantity'` | The quantity of each product (item) per transaction |
| `'UnitPrice'` | Product price per unit |
| `'CustomerID'` | A 5-digit number uniquely assigned to each customer |
| `'Country'` | The name of the country where each customer resides |
| `'InvoiceDate'` | The day and time when each transaction was generated `"MM/DD/YYYY"` |
| `'Year'` | The year when each transaction was generated |
| `'Month'` | The month when each transaction was generated |
| `'Week'` | The week when each transaction was generated (`1`-`52`) |
| `'Day'` | The day of the month when each transaction was generated (`1`-`31`) |
| `'DayOfWeek'` | The day of the weeke when each transaction was generated <br>(`0` = Monday, `6` = Sunday) |

In [38]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.sql.functions import col, dayofmonth, month, year,  to_date, to_timestamp, weekofyear, dayofweek, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Initialize Spark session
my_spark = SparkSession.builder.appName("SalesForecast").getOrCreate()

# Importing sales data
sales_data = my_spark.read.csv(
    "Online Retail.csv", header=True, inferSchema=True, sep=",")

# Convert InvoiceDate to datetime 
sales_data = sales_data.withColumn("InvoiceDate", to_date(
    to_timestamp(col("InvoiceDate"), "d/M/yyyy H:mm")))

In [39]:
# STEP 1: Group the data into daily intervals, using the .groupBy() method on the following columns: "Country", "StockCode", "InvoiceDate", "Year", "Month", "Day", "Week", "DayOfWeek", saving as a new variable. Use .agg() on the grouped data, which accepts a dictionary with column names as keys and the type of calculation as values. With this approach, get the "sum" of the "Quantity" column and the "avg" of "UnitPrice".
sales_data_grouped = sales_data.groupBy("Country", "StockCode", "InvoiceDate", "Year", "Month", "Day", "Week", "DayOfWeek").agg({"Quantity": "sum", "UnitPrice": "avg"})

# To keep the target name consistent, rename the aggregated "sum(Quantity)" column to "Quantity" using the .withColumnRenamed() method.
sales_data_grouped = sales_data_grouped.withColumnRenamed('Quantity', "sum(Quantity)").show()

+--------------+---------+-----------+----+-----+---+----+---------+--------------+-------------+
|       Country|StockCode|InvoiceDate|Year|Month|Day|Week|DayOfWeek|avg(UnitPrice)|sum(Quantity)|
+--------------+---------+-----------+----+-----+---+----+---------+--------------+-------------+
|United Kingdom|    22912| 2010-01-12|2010|    1| 12|   2|        1|          4.95|            3|
|        France|    22659| 2010-01-12|2010|    1| 12|   2|        1|          1.95|           24|
|United Kingdom|    21544| 2010-01-12|2010|    1| 12|   2|        1|          0.85|           12|
|United Kingdom|    21098| 2010-01-12|2010|    1| 12|   2|        1|          1.25|           16|
|        Norway|    85150| 2010-01-12|2010|    1| 12|   2|        1|          2.55|           12|
|United Kingdom|    22195| 2010-01-12|2010|    1| 12|   2|        1|          1.65|            5|
|United Kingdom|    21668| 2010-01-12|2010|    1| 12|   2|        1|          1.25|           18|
|United Kingdom|    

In [45]:
# STEP 2: Splitting data into 2 datasets
# Sales data till 2011-09-25 (incl.)
train_data = sales_data.filter(col("InvoiceDate") <= lit("2011-09-25"))
# Sales data after 2011-09-25
test_data = sales_data.filter(col("InvoiceDate") > lit("2011-09-25"))


In [41]:
# STEP 3: building regression model
# Use StringIndexer() to encode your categorical columns ('Country' and 'StockCode'), saving as country_indexer and stock_code_indexer. When doing this, chain the String Indexer's .setHandleInvalid() method, setting to "keep"

country_indexer = StringIndexer(inputCol="Country", outputCol="CountryIndex").setHandleInvalid("keep")
stock_code_indexer = StringIndexer(inputCol="StockCode", outputCol="StockCodeIndex").setHandleInvalid("keep")

# combine all selected features into a vector, providing a list of features to the inputCols argument and setting the outputCol to a name for the features

assembler = VectorAssembler(inputCols=["CountryIndex", "StockCodeIndex"], outputCol="features")

# RandomForestModel

rf = RandomForestRegressor(featuresCol="features", labelCol="Quantity", maxBins=4000)

# Pipeline - combining all steps

pipeline = Pipeline(stages= [country_indexer, stock_code_indexer, assembler, rf])

# model

model = pipeline.fit(train_data)

25/04/15 11:46:07 WARN DAGScheduler: Broadcasting large task binary with size 1106.4 KiB


25/04/15 11:46:08 WARN DAGScheduler: Broadcasting large task binary with size 1855.8 KiB


25/04/15 11:46:11 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB


In [42]:
# STEP 4 Evaluating the model (MAE)

test_predictions = model.transform(test_data)

test_predictions = test_predictions.withColumn("prediction", col("prediction").cast("double"))

# Calculate MAE

mae_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol='Quantity', metricName='mae')
mae = mae_evaluator.evaluate(test_predictions)
print(mae)

5.410738521647606


In [43]:
# STEP 5: Identify the quantity sold at specific week
quantity_sold_w39 = int(test_predictions.groupBy(['Year', 'Week']).agg({'prediction': 'sum'}).filter((col('Week')==39) & (col('Year')==2011)).select('sum(prediction)').collect()[0][0])
print(quantity_sold_w39)

87346
